# Описание задания

Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

Сформируйте параллельный ансамбль из:
* CatBoost,
* градиентного бустинга,
* XGBoost
* LightGBM.

Используйте лучшие гиперпараметры, подобранные ранее, или найдите их через перекрестную проверку.

Итоговое решение рассчитайте на основании самого точного предсказания класса у определенной модели ансамбля: выберите для каждого класса модель, которая предсказывает его лучше всего.

Проведите расчеты и выгрузите результат в виде submission.csv

Данные:
* video.ittensive.com/machine-learning/prudential/train.csv.gz
* video.ittensive.com/machine-learning/prudential/test.csv.gz
* video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz

Итоговый файл с кодом (.py или .ipynb) выложите в github с портфолио.

# Решение задания

In [1]:
GRAIN = 11
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import preprocessing
from etl_utils import reduce_mem_usage


def data_preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df['Product_Info_2_1'] = df['Product_Info_2'].str.slice(0, 1)
    df['Product_Info_2_2'] = pd.to_numeric(df['Product_Info_2'].str.slice(1, 2))
    df = df.drop('Product_Info_2', axis='columns')

    onehot_df = pd.get_dummies(df['Product_Info_2_1'])
    onehot_df.columns = ['Product_Info_2_1' + column for column in onehot_df.columns]
    df = pd.merge(left=df, right=onehot_df, left_index=True, right_index=True).drop('Product_Info_2_1', axis=1).fillna(-1)
    del onehot_df

    if 'Response' in df.columns:
        df['Response'] = df['Response'] - 1
    return df


def f1_score_for_class(class_id: int, y_true, y_pred):
    return f1_score(
        pd.Series(y_true).map(lambda value: 1 if value == class_id else 0),
        pd.Series(y_pred).map(lambda value: 1 if value == class_id else 0)
    )


def model_estimates(y_true, y_pred) -> list:
    '''
    Формирует f1 оценки, количество оценок соответствует количеству классов в y_true.
    y_true - вектор эталонных ответов
    y_pred - вектор предсказаний классов
    
    Возвращает вектор чисел float
    '''
    result = list()
    for i in range(len(y_train_test.unique())):
        result.append(f1_score_for_class(i, y_true, y_pred))    
    return result

### Загрузка обучающей выборки и определение набора признаков для дальнейшего моделирования

In [2]:
train_df = data_preprocess(pd.read_csv('https://video.ittensive.com/machine-learning/prudential/train.csv.gz'))

columns_groups = ['Insurance_History', 'InsurеdInfo', 'Medical_Keyword', 'Family_Hist', 'Medical_History', 'Product_Info']
features = ['Wt', 'Ht', 'Ins_Age', 'BMI']
for cg in columns_groups:
    features.extend(train_df.columns[train_df.columns.str.startswith(cg)])
print(features)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация и разделение данных

In [3]:
scaler = preprocessing.StandardScaler()
train_df_scaled = pd.DataFrame(scaler.fit_transform(train_df[features]))
features_in_scaled = train_df_scaled.columns
train_df_scaled['Response'] = train_df['Response']
train_df_scaled = reduce_mem_usage(train_df_scaled)

train_train_df, train_test_df = train_test_split(train_df_scaled, test_size=.2, random_state=GRAIN)
print(train_train_df.shape, train_test_df.shape)

Потребление памяти меньше на 40.49 Мб (-75.1%)
(47504, 119) (11877, 119)


### Матрица, в которую буду сохранять f1 оценки прогнозов каждой моделью каждого класса
(для последующего выбора моделей)
* строки - это целевые для предсказания классы
* столбцы - это модели

На пересечении строк и столбцов значение f1_score для пары конкретных модели и класса

In [4]:
f1_scores_df = pd.DataFrame(index=[sorted(train_df['Response'].unique())])

### Построение моделей

Использую лучшие гиперпараметры, подобранные ранее, так как их подбор через перекрестную проверку занял существенное время.

In [5]:
x_train_train = train_train_df[features_in_scaled]
y_train_train = train_train_df['Response']

x_train_test = train_test_df[features_in_scaled]
y_train_test = train_test_df['Response']

x_train = train_df_scaled[features_in_scaled]
y_train = train_df_scaled['Response']

#### CatBoost
Основные гиперпараметры беру из тех, что подобраны ранее (времени на это затрачено ощутимо).

In [6]:
%%time
from catboost import Pool, CatBoostClassifier
model_CatBoost = CatBoostClassifier(
    random_seed=GRAIN, loss_function='MultiClass', bootstrap_type='MVS', custom_metric='WKappa',
    learning_rate=.56, depth=7, l2_leaf_reg=4, iterations=100
).fit(Pool(data=x_train_train, label=y_train_train))

0:	learn: 1.5308178	total: 200ms	remaining: 19.8s
1:	learn: 1.3929447	total: 261ms	remaining: 12.8s
2:	learn: 1.3382318	total: 335ms	remaining: 10.8s
3:	learn: 1.3114882	total: 395ms	remaining: 9.47s
4:	learn: 1.2898592	total: 462ms	remaining: 8.78s
5:	learn: 1.2793855	total: 522ms	remaining: 8.18s
6:	learn: 1.2686422	total: 578ms	remaining: 7.68s
7:	learn: 1.2501712	total: 637ms	remaining: 7.32s
8:	learn: 1.2412142	total: 698ms	remaining: 7.06s
9:	learn: 1.2343899	total: 759ms	remaining: 6.83s
10:	learn: 1.2250090	total: 817ms	remaining: 6.61s
11:	learn: 1.2124148	total: 876ms	remaining: 6.42s
12:	learn: 1.2087139	total: 937ms	remaining: 6.27s
13:	learn: 1.2037673	total: 998ms	remaining: 6.13s
14:	learn: 1.1999816	total: 1.06s	remaining: 6.01s
15:	learn: 1.1963732	total: 1.12s	remaining: 5.87s
16:	learn: 1.1884244	total: 1.18s	remaining: 5.78s
17:	learn: 1.1840390	total: 1.24s	remaining: 5.66s
18:	learn: 1.1781063	total: 1.3s	remaining: 5.55s
19:	learn: 1.1736369	total: 1.36s	remainin

In [7]:
f1_scores_df['CatBoost'] = model_estimates(y_train_test, model_CatBoost.predict(Pool(data=x_train_test)).flatten())

Обучение модели на полной тренировочной выборке

In [8]:
%%time
model_CatBoost = CatBoostClassifier(
    random_seed=GRAIN, loss_function='MultiClass', bootstrap_type='MVS', custom_metric='WKappa',
    learning_rate=.56, depth=7, l2_leaf_reg=4, iterations=100
).fit(Pool(data=x_train, label=y_train))

0:	learn: 1.5197487	total: 84.4ms	remaining: 8.36s
1:	learn: 1.3915691	total: 158ms	remaining: 7.74s
2:	learn: 1.3435663	total: 227ms	remaining: 7.33s
3:	learn: 1.3129286	total: 298ms	remaining: 7.14s
4:	learn: 1.2843400	total: 374ms	remaining: 7.1s
5:	learn: 1.2727900	total: 446ms	remaining: 6.98s
6:	learn: 1.2603072	total: 517ms	remaining: 6.86s
7:	learn: 1.2452974	total: 594ms	remaining: 6.83s
8:	learn: 1.2330872	total: 672ms	remaining: 6.79s
9:	learn: 1.2291206	total: 760ms	remaining: 6.84s
10:	learn: 1.2217220	total: 841ms	remaining: 6.81s
11:	learn: 1.2090635	total: 918ms	remaining: 6.73s
12:	learn: 1.2052953	total: 999ms	remaining: 6.69s
13:	learn: 1.2014075	total: 1.08s	remaining: 6.64s
14:	learn: 1.1961090	total: 1.16s	remaining: 6.57s
15:	learn: 1.1910428	total: 1.24s	remaining: 6.53s
16:	learn: 1.1884284	total: 1.32s	remaining: 6.47s
17:	learn: 1.1849168	total: 1.41s	remaining: 6.43s
18:	learn: 1.1806464	total: 1.51s	remaining: 6.43s
19:	learn: 1.1774053	total: 1.6s	remainin

#### Градиентный бустинг
Основные гиперпараметры беру из тех, что подобраны ранее (времени на это затрачено ощутимо).

In [9]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
model_GradientBoosting = GradientBoostingClassifier(
    random_state=GRAIN, max_depth=17, n_estimators=76, max_features=27, min_samples_leaf=20
).fit(x_train_train, y_train_train)

CPU times: total: 2min 2s
Wall time: 2min 3s


In [10]:
f1_scores_df['GradientBoosting'] = model_estimates(y_train_test, model_GradientBoosting.predict(x_train_test))

Обучение модели на полной тренировочной выборке

In [11]:
%%time
model_GradientBoosting.fit(x_train, y_train)

CPU times: total: 2min 36s
Wall time: 2min 36s


GradientBoostingClassifier(max_depth=17, max_features=27, min_samples_leaf=20,
                           n_estimators=76, random_state=11)

#### XGBoost
Основные гиперпараметры беру из тех, что подобраны ранее (времени на это затрачено ощутимо).

In [12]:
%%time
from xgboost import XGBClassifier
model_XGBoost = XGBClassifier(
    max_depth=15, n_estimators=76, max_features=27, min_samples_leaf=19
).fit(x_train_train, y_train_train)

[19:18:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features", "min_samples_leaf" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


CPU times: total: 10min 48s
Wall time: 43.6 s


In [13]:
f1_scores_df['XGBoost'] = model_estimates(y_train_test, model_XGBoost.predict(x_train_test))

Обучение модели на полной тренировочной выборке

In [14]:
%%time
model_XGBoost.fit(x_train, y_train)

[19:19:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features", "min_samples_leaf" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


CPU times: total: 14min 13s
Wall time: 59.8 s


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=15, max_features=27, max_leaves=0,
              min_child_weight=1, min_samples_leaf=19, missing=nan,
              monotone_constraints='()', n_estimators=76, n_jobs=0,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto', ...)

#### LightGBM
Основные гиперпараметры беру из тех, что подобраны ранее (времени на это затрачено ощутимо).

In [15]:
%%time
import lightgbm as lgb
model_LGBM = lgb.LGBMRegressor(
    random_state=GRAIN, n_estimators=1000, objective='multiclass', num_class=8,
    max_depth=16, min_child_samples=18, num_leaves=36
).fit(x_train_train, y_train_train)

CPU times: total: 4min 42s
Wall time: 20.7 s


In [16]:
def to_class(row: pd.Series):
    return np.argmax(row)

f1_scores_df['LightGBM'] = model_estimates(
    y_train_test, pd.DataFrame(model_LGBM.predict(x_train_test)).apply(to_class, axis='columns')
)

Обучение модели на полной тренировочной выборке

In [17]:
%%time
model_LGBM.fit(x_train, y_train)

CPU times: total: 4min 24s
Wall time: 18.8 s


LGBMRegressor(max_depth=16, min_child_samples=18, n_estimators=1000,
              num_class=8, num_leaves=36, objective='multiclass',
              random_state=11)

### Выберите для каждого класса модель, которая предсказывает его лучше всего.

In [18]:
f1_scores_df

,CatBoost,GradientBoosting,XGBoost,LightGBM
0,0.266263,0.293069,0.272867,0.283325
1,0.303917,0.316379,0.285441,0.313779
2,0.396450,0.418879,0.432277,0.356688
3,0.664662,0.647239,0.642750,0.622642
4,0.567634,0.569307,0.571429,0.571715
5,0.506679,0.515718,0.521470,0.510097
6,0.454967,0.458071,0.462524,0.457689
7,0.779388,0.779010,0.778616,0.782193


In [19]:
def test_row(row: pd.Series):
    return np.argmax(row)


opitmal_models = f1_scores_df.apply(test_row, axis='columns').to_list()
for class_id in range(len(opitmal_models)):
    print('Для класса', class_id, 'оптимальна модель', f1_scores_df.columns[opitmal_models[class_id]])

Для класса 0 оптимальна модель GradientBoosting
Для класса 1 оптимальна модель GradientBoosting
Для класса 2 оптимальна модель XGBoost
Для класса 3 оптимальна модель CatBoost
Для класса 4 оптимальна модель LightGBM
Для класса 5 оптимальна модель XGBoost
Для класса 6 оптимальна модель XGBoost
Для класса 7 оптимальна модель LightGBM


### Загрузка отложенной выборки и приведение данных к формату обущающей выборки

In [20]:
test_df = pd.read_csv('https://video.ittensive.com/machine-learning/prudential/test.csv.gz')
test_df = reduce_mem_usage(data_preprocess(test_df))
test_transformed_df = reduce_mem_usage(pd.DataFrame(scaler.transform(test_df[features])))
test_transformed_df.info()

Потребление памяти меньше на 16.34 Мб (-84.9%)
Потребление памяти меньше на 13.35 Мб (-75.0%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19765 entries, 0 to 19764
Columns: 118 entries, 0 to 117
dtypes: float16(118)
memory usage: 4.4 MB


### Предсказание данных

In [62]:
def predict(x: pd.Series):
    x = x.to_numpy().reshape(1, -1)
    prediction = model_XGBoost.predict(x).flatten()[0]
    if prediction == 2 or prediction == 5 or prediction == 6:
        return prediction
    
    prediction = pd.DataFrame(model_LGBM.predict(x)).apply(to_class, axis='columns')[0]
    if prediction == 4 or prediction == 7:
        return prediction
        
    prediction = model_GradientBoosting.predict(x)[0]
    if prediction == 0 or prediction == 1:
        return prediction

    return model_CatBoost.predict(data=x).flatten()[0]

In [ ]:
%%time
test_transformed_df['Response'] = test_transformed_df.apply(predict, axis='columns')

В моделе порядковые значения классов от 0 до 7, в ответе они должны быть от 1 до 8 - преобразую к этому формату

In [64]:
test_transformed_df['Response'] += 1

### Формирование результатов

In [63]:
submission = pd.read_csv('https://video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz')
submission.head()

,Id,Response
0,1,8
1,3,8
2,4,8
3,9,8
4,12,8


In [65]:
submission['Response'] = test_transformed_df['Response'].astype('int8')
submission.head()

,Id,Response
0,1,1
1,3,8
2,4,7
3,9,7
4,12,8


### Выгрузка результатов

In [66]:
submission.to_csv('submission', index=False)